In [ ]:
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_openai import ChatOpenAI

@tool
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

model = ChatOpenAI(model="gpt-5-mini",
                   api_key="YOUR-API-KEY", 
                   temperature=0)

agent = create_agent(
    model,
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

print(result)


{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='baa96daf-e4e6-488c-9d29-d3151d458508'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 140, 'total_tokens': 228, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cpfld5YOT0OYSkpCDDcC7VtWZJdvO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b4782-da6c-7aa3-b663-578782de8a03-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_VFUR9G6oBl0kdFB0wrsZPb3X', 'type': 'tool_call'}], usage_metadata={'input_tokens': 140, 'output_tokens': 88, 'total_tokens':

# Build a real world agent

The system prompt defines your agent’s role and behavior. Keep it specific and actionable:

In [2]:
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

Tools let a model interact with external systems by calling functions you define. Tools can depend on runtime context and also interact with agent memory.

In [3]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

set up languange model

In [ ]:
# from langchain.chat_models import init_chat_model

# model = init_chat_model(
#     "claude-sonnet-4-5-20250929",
#     temperature=0.5,
#     timeout=10,
#     max_tokens=1000
# )

model = ChatOpenAI(model="gpt-5-mini",
        api_key="YOUR-API-KEY", 
        temperature=0.5,
        timeout=60,
        max_tokens=1000         
                  )

Define Response Format

In [5]:
from dataclasses import dataclass

# We use a dataclass here, but Pydantic models are also supported.
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

Add memory to your agent to maintain state across interactions. This allows the agent to remember previous conversations and context.

In [6]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

In [ ]:
from langchain.agents.structured_output import ToolStrategy

agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )

# Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="You're 'thund-erfully' welcome! It's always a 'breeze' to help you stay 'current' with the weather. I'm just 'cloud'-ing around waiting to 'shower' you with more forecasts whenever you need them. Have a 'sun-sational' day in the Florida sunshine!",
#     weather_conditions=None
# )

In [ ]:
from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str

agent = create_agent(
    model="gpt-4o-mini",
    tools=[search_tool],
    response_format=ToolStrategy(ContactInfo)
)

result = agent.invoke({
    "messages": [{"role": "user", 
                  "content": "Extract contact info from: John Doe, 
                  john@example.com, (555) 123-4567"}]
})

result["structured_response"]
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

In [ ]:
result

In [1]:
# Dynamic System Prompt
# !pip install langchain-tavily

In [8]:
# # from langchain.tools.tavily_search import TavilySearchResults
# from langchain_community.tools.tavily_search import TavilySearchResults


# web_search = TavilySearchResults()


In [5]:
from typing import TypedDict

from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest


class Context(TypedDict):
    user_role: str

@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on user role."""
    user_role = request.runtime.context.get("user_role", "user")
    base_prompt = "You are a helpful assistant."

    if user_role == "expert":
        return f"{base_prompt} Provide detailed technical responses."
    elif user_role == "beginner":
        return f"{base_prompt} Explain concepts simply and avoid jargon."

    return base_prompt

agent = create_agent(
    model="gpt-5-mini",
    middleware=[user_role_prompt],
    context_schema=Context
)

# The system prompt will be set dynamically based on context
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Explain machine learning"}]},
    context={"user_role": "beginner"}
)

In [8]:
result

{'messages': [HumanMessage(content='Explain machine learning', additional_kwargs={}, response_metadata={}, id='66f29c35-d2f8-43ff-bfde-a009ec16d0d5'),
  AIMessage(content='Machine learning (ML) is a way for computers to learn patterns from data so they can make predictions or decisions without being explicitly programmed for each specific task.\n\nShort analogy\n- Teaching a child: instead of writing rules for every possible situation, you show examples (pictures of cats and dogs). Over time the child learns to recognize a cat from patterns in the examples. ML does the same with data.\n\nHow it works (high level)\n1. Collect data: examples that show the input and (often) the desired output.\n2. Choose a model: a flexible program structure that can represent relationships in the data (simple line, tree, neural network, etc.).\n3. Train the model: adjust the model’s internal settings so its outputs match the examples as well as possible.\n4. Evaluate: test the model on new data it hasn’t

In [1]:
import os
os.getenv("OPENAI_API_KEY")

In [9]:
#Structured output

In [ ]:
from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str

agent = create_agent(
    model="gpt-4o-mini",
    # tools=[search_tool],
    response_format=ToolStrategy(ContactInfo)
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

: 

In [ ]:
export OPENAI_API_KEY="YOUR-API-KEY"